![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Medical_Language_Models/April_2025/Notebooks/Dicom_Metadata_Only.ipynb)

<!-- ========================================================= -->
<!--        John Snow Labs - Package Installation Guide         -->
<!--        JupyterLab Single Markdown Cell (HTML Content)      -->
<!-- ========================================================= -->

<h1>Package Installation</h1>

<!-- Link to official GitHub repository -->
<p>
  Official Repository:
  <a href="https://github.com/JohnSnowLabs/johnsnowlabs" target="_blank">
    https://github.com/JohnSnowLabs/johnsnowlabs
  </a>
</p>

<!-- License setup instructions -->
<p>
  Keep your <strong>license keys</strong> in a JSON file and point to it using the
  <code>json_license_path</code> argument when starting the Spark session.
</p>

<!-- Visual NLP configuration note -->
<p>
  Set <code>visual=True</code> while starting the Spark session to install and make
  <strong>Visual NLP libraries</strong> available.
</p>

<!-- Restart note -->
<p>
  ⚠️ <strong>Important:</strong> After installing the library, make sure to
  <strong>RESTART your session</strong> before running Spark again.
</p>

<!-- End of notebook cell -->

In [ ]:
!pip install -q johnsnowlabs

In [ ]:
from johnsnowlabs import nlp, visual, medical

nlp.install(refresh_install=True, visual=True, json_license_path="./spark_nlp_for_healthcare_spark_ocr_10538.json")

In [ ]:
# RESTART SESSION!!!

In [1]:
from johnsnowlabs import visual, nlp

spark = nlp.start(visual=True, hardware_target="gpu", json_license_path="./spark_nlp_for_healthcare_spark_ocr_10538.json")

spark

👌 License info detected in file ./spark_nlp_for_healthcare_spark_ocr_10538.json


25/10/22 20:25:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


👌 Launched gpu optimized session with with: 🚀Spark-NLP==6.1.3, 💊Spark-Healthcare==6.1.1, 🕶Spark-OCR==6.1.0, running on ⚡ PySpark==3.4.0


<h1>Dicom Metadata DeIdentification</h1>

<!-- Notebook purpose -->
<p>
  This notebook focuses on the <strong>DICOM Metadata De-Identification</strong> process — the anonymization of
  Protected Health Information (PHI) stored within DICOM headers and nested tag structures.
  It demonstrates how <strong>Visual NLP</strong> transformers can be applied
  to sanitize sensitive metadata fields across large DICOM datasets while maintaining UID consistency
  and referential integrity.
</p>

<!-- Stages list -->
<h2>Concepts and Components Covered in this Notebook</h2>
<ul>
  <li>DicomToMetadata</li>
  <li>DicomMetadataDeidentifier</li>
</ul>

In [2]:
# 📦 Spark OCR Imports
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import *
from sparkocr.schemas import BinarySchema

# ⚡ Spark NLP Core
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

# 🔗 Spark ML
from pyspark.ml import Pipeline, PipelineModel
import pyspark.sql.functions as F

# 🧩 Spark NLP for Healthcare (JSL)
import sparknlp_jsl
from sparknlp_jsl.annotator import *
from sparkocr.base import LightPipeline

from pyspark.sql.functions import *
from pyspark.sql.types import *
from urllib.parse import urlparse
from IPython.display import display, Markdown
from PIL import Image, ImageDraw, ImageFont
import pkg_resources
import pandas as pd
import time

In [3]:
def compare_dicom_metadata(dicom_original_metadata, dicom_redacted_metadata):
    """
    Compare Original and Redacted Metadata
    Add a new column denoting if value has changed
    Returns pandas dataframe
    """
    metadata_mapping = []

    for key in dicom_original_metadata.keys():
        changed = dicom_original_metadata[key] != dicom_redacted_metadata[key]
        metadata_mapping.append([key,dicom_original_metadata[key],dicom_redacted_metadata[key], changed])

    metadata_mapping_df = pd.DataFrame(metadata_mapping, columns=["Key", "Original Value", "Redacted Value", "Value Changed"])

    return metadata_mapping_df

<h2>Extract Metadata from Dicom Object</h2>

In [4]:
dicom_to_metadata = DicomToMetadata() \
    .setInputCol("content") \
    .setOutputCol("metadata")

In [5]:
df = spark.read.format("binaryFile").load("./data/visual/dicom/David_Douglas.dcm")

result = dicom_to_metadata.transform(df).select("metadata").collect()[0].asDict()["metadata"]

json_result = json.loads(result)

json_result

/usr/local/lib/python3.12/dist-packages/sparkocr/transformers/dicom/dicom_metadata_deidentifier.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
                                                                                

{'ImageType': "['DERIVED', 'PRIMARY', 'POST_PROCESSED', 'RT', '', '', '', '', '100000']",
 'SOPClassUID': '1.2.840.10008.5.1.4.1.1.1',
 'SOPInstanceUID': '2.25.112784503178059210578740147414000844278',
 'StudyDate': '20130713',
 'SeriesDate': '20130713',
 'AcquisitionDate': '20130713',
 'ContentDate': '20130713',
 'StudyTime': '085957',
 'SeriesTime': '090020.000',
 'AcquisitionTime': '090020.408',
 'ContentTime': '090143.249',
 'AccessionNumber': '20130714E864535',
 'Modality': 'CR',
 'Manufacturer': 'FUJIFILM Corporation',
 'InstitutionName': 'Scott Community Hospital',
 'InstitutionAddress': '334 Michael Manor Sarahview, PA 56560',
 "ReferringPhysician'sName": 'HUGHES^KATHLEEN',
 "ReferringPhysician'sAddress": '0544 Green Inlet Jeffreyland, HI 66060',
 'StudyDescription': 'XR CHEST AP PORTABLE for Douglas Davidson',
 'ProcedureCodeSequence': [{'CodeValue': 'CHESTAPPAP',
   'CodingSchemeDesignator': 'SECTRA',
   'CodeMeaning': 'XR CHEST AP PORTABLE'}],
 "PerformingPhysician'sName": '

<h2>Run Deidentify Dicom Metadata and Extract Results</h2>

In [6]:
csv_path = pkg_resources.resource_filename('sparkocr', "resources/ocr/dicom/default_dicom_strategy.csv")

strategy_csv = pd.read_csv(csv_path)

strategy_csv.head(10)

,Tags,VR,Name,Status,Action,Repeatable
0,"(0002,0100)",UI,Private Information Creator UID,NaN,hashId,NaN
1,"(0002,0102)",OB,Private Information,NaN,hashId,NaN
2,"(0004,1130)",CS,File-set ID,NaN,hashId,NaN
3,"(0004,1141)",CS,File-set Descriptor File ID,NaN,hashId,NaN
4,"(0004,1432)",UI,Private Record UID,NaN,hashId,NaN
5,"(0004,1500)",CS,Referenced File ID,NaN,hashId,NaN
6,"(0008,0012)",DA,Instance Creation Date,NaN,shiftDateByRandomNbOfDays,NaN
7,"(0008,0014)",UI,Instance Creator UID,NaN,hashId,NaN
8,"(0008,0020)",DA,Study Date,NaN,shiftDateByFixedNbOfDays,112
9,"(0008,0021)",DA,Series Date,NaN,shiftDateByRandomNbOfDays,NaN


In [7]:
dicom_deidentifier = DicomMetadataDeidentifier() \
    .setInputCols(["content"]) \
    .setOutputCol("dicom_metadata_cleaned") \
    .setRemovePrivateTags(False) \
    .setPlaceholderText("<hidden>") \
    .setStrategyFile(csv_path)

In [8]:
df = spark.read.format("binaryFile").load("./data/visual/dicom/David_Douglas.dcm")

result = dicom_deidentifier.transform(df).write \
  .format("binaryFormat") \
  .option("type", "dicom") \
  .option("field", "dicom_metadata_cleaned") \
  .option("nameField", "fileName") \
  .option("extension", "dcm") \
  .option("prefix", "de-id-") \
  .mode("overwrite") \
  .save("./data/result_metadata/")

20:25:53, INFO Run DicomMetadataDeidentifier
20:25:53, INFO DEBUG: tag:(2, 256), vr:UI, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(2, 258), vr:OB, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(4, 4400), vr:CS, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(4, 4417), vr:CS, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(4, 5170), vr:UI, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(4, 5376), vr:CS, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(8, 18), vr:DA, action:shiftDateByRandomNbOfDays, option:None
20:25:53, INFO DEBUG: tag:(8, 20), vr:UI, action:hashId, option:None
20:25:53, INFO DEBUG: tag:(8, 32), vr:DA, action:shiftDateByFixedNbOfDays, option:112
20:25:53, INFO Run DicomMetadataDeidentifier:vr:DA, action:shiftDateByFixedNbOfDays
20:25:53, INFO DEBUG: tag:(8, 33), vr:DA, action:shiftDateByRandomNbOfDays, option:None
20:25:53, INFO Run DicomMetadataDeidentifier:vr:DA, action:shiftDateByRandomNbOfDays
20:25:53, INFO DEBUG: tag:(8, 3

<h2>Run DicomToMetadata and Extract Results</h2>

In [9]:
df = spark.read.format("binaryFile").load("./data/result_metadata/*.dcm")

deid_result = dicom_to_metadata.transform(df).select("metadata").collect()[0].asDict()["metadata"]

json_deid_result = json.loads(deid_result)

json_deid_result

{'ImageType': "['DERIVED', 'PRIMARY', 'POST_PROCESSED', 'RT', '', '', '', '', '100000']",
 'SOPClassUID': '1.2.840.10008.5.1.4.1.1.1',
 'SOPInstanceUID': '2.25.112784503178059210578740147414000844278',
 'StudyDate': '20131102',
 'SeriesDate': '20130819',
 'AcquisitionDate': '20130616',
 'ContentDate': '20130519',
 'StudyTime': '085957',
 'SeriesTime': '090020.000',
 'AcquisitionTime': '090020.408',
 'ContentTime': '090143.249',
 'AccessionNumber': '2.25.104661169016722362372224050231930259815',
 'Modality': 'CR',
 'Manufacturer': 'FUJIFILM Corporation',
 'InstitutionName': '<hidden>',
 'InstitutionAddress': '<hidden>',
 "ReferringPhysician'sName": '<hidden>',
 "ReferringPhysician'sAddress": '<hidden>',
 'StudyDescription': 'XR CHEST AP PORTABLE for Douglas Davidson',
 'ProcedureCodeSequence': [{'CodeValue': 'CHESTAPPAP',
   'CodingSchemeDesignator': 'SECTRA',
   'CodeMeaning': 'XR CHEST AP PORTABLE'}],
 "PerformingPhysician'sName": '<hidden>',
 'DerivationDescription': 'G1.0e#1.60+0.20

<h3>Compare the Tags with Changes</h3>

In [10]:
pd_df = compare_dicom_metadata(json_result, json_deid_result)
pd_df[pd_df["Value Changed"] == True].head(50)

,Key,Original Value,Redacted Value,Value Changed
3,StudyDate,20130713,20131102,True
4,SeriesDate,20130713,20130819,True
5,AcquisitionDate,20130713,20130616,True
6,ContentDate,20130713,20130519,True
11,AccessionNumber,20130714E864535,2.25.104661169016722362372224050231930259815,True
14,InstitutionName,Scott Community Hospital,<hidden>,True
15,InstitutionAddress,"334 Michael Manor Sarahview, PA 56560",<hidden>,True
16,ReferringPhysician'sName,HUGHES^KATHLEEN,<hidden>,True
17,ReferringPhysician'sAddress,"0544 Green Inlet Jeffreyland, HI 66060",<hidden>,True
20,PerformingPhysician'sName,BROWN^PETER,<hidden>,True


In [11]:
import shutil
shutil.rmtree("./data/result_metadata/")